In [1]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [2]:
def contBatActions2(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def contBatActions(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    if x[0]-lowBound < 0.001:
        x[0]=lowBound
    if upBound-x[0] < 0.001:
        x[0]=upBound
    dim = len(x)
    bat_action = x[0]-bat_t0
    bat_action=float(bat_action)
    if abs(bat_action) <= 0.001:
        bat_action=0.0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        bat_action=float(bat_action)
        if abs(bat_action) <= 0.001:
            bat_action=0.0
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def costFunctionBatteryLifetime3(x,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):

    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    batActionTotal=0
    
    numCharges=0
    numDischarges=0
    batLifeLoss=0
    dischAction=0
    DOD=0
    dischFinish=0
    
    charOrDisch2 = battery.action
    contChar=0
    contDisch=0
    
    numCharChanges=0
    numBatNullState=0
    dim = len(x)
    bat_action = x[0]-bat_t0
    
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if bat_action == 0:
        numBatNullState+=1
        
    if charOrDisch2 == 1 and bat_action < 0:
        charOrDisch2=-1
        numCharChanges+=1 
        numDischarges+=1
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=bat_action
        dischFinish=0
        
    elif charOrDisch2 == -1 and bat_action < 0:
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=(bat_action_t0+bat_action)
        dischFinish=0
            
    elif charOrDisch2 == -1 and bat_action > 0:
        charOrDisch2=1
        numCharChanges+=1
        numCharges+=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    batActionTotal+=abs(bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
            
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if bat_action == 0:
            numBatNullState+=1
            
        if charOrDisch2 == 1 and bat_action < 0:
            charOrDisch2=-1
            numCharChanges+=1 
            numDischarges+=1
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction=bat_action
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action < 0:
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction+=(bat_action)
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action > 0:
            charOrDisch2=1
            numCharChanges+=1
            numCharges+=1
            batLifeLoss+=(abs(dischAction)*DOD)
            dischFinish=1
            
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
        batActionTotal+=abs(bat_action)

    if dischFinish==0:
        batLifeLoss+=(abs(dischAction)*DOD)
    if numDischarges>0:
        batLifeLoss=batLifeLoss#/numDischarges
    else:
        batLifeLoss=0
    fitness = (const1*grid_sum)+(const2*numBatNullState)+(const3*batLifeLoss)
    return fitness, charOrDisch2, numCharges, numDischarges, grid_sum, numCharChanges, batLifeLoss

In [3]:
class Battery:
    def __init__(self,cap,soc,act,contAct,numContChar,numContDisch,numCharges,numDischarges,numBatNullState,numCharChanges):
        self.capacity=cap
        self.SOC=soc
        self.action=act
        self.contAct=contAct
        self.numContChar=numContChar
        self.numContDisch=numContDisch
        self.numCharges=numCharges
        self.numDischarges=numDischarges
        self.numBatNullState=numBatNullState
        self.numCharChanges=numCharChanges
        self.batLifeLoss=0
        self.avgDOD=0
        self.sumDOD=0
        self.avgDODPerDisch=0
        self.sumDODPerDisch=0
        self.avgSOC=0
        self.sumSOC=0
        self.avgDischAction=0
        self.sumDischAction=0
        self.totalBat=0

In [4]:
class Particle:
    def __init__(self,x0,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):
        self.position_i=[]          # particle position
        self.velocity_i=[]          # particle velocity
        self.pos_best_i=[]          # best position individual
        self.err_best_i=-1          # best error individual
        self.err_i=-1               # error individual
        self.dem=demand
        self.pv_gen=pv_gen
        self.bat_t0=bat_t0                        #som pridal
        self.bat_action_t0=bat_action_t0          #som pridal
        self.elprices=elprices                    #som pridal
        self.dist_best_i=-1                       #som pridal
        self.sthr=0.2
        
        self.battery = battery       #pridal
        
        self.charOrDisch2_i = 0      #pridal
        self.numBatNullState_i = 0   #pridal
        self.numCharChanges_i = 0    #pridal
        self.numCharges_i = 0        #pridal
        self.numDischarges_i = 0     #pridal
        
        self.charOrDisch2Best_i = 0      #pridal
        self.numBatNullStateBest_i = 0   #pridal
        self.numCharChangesBest_i = 0    #pridal
        self.numChargesBest_i = 0        #pridal
        self.numDischargesBest_i = 0     #pridal
        
        self.gridPrice_i = 0       #pridal
        self.gridPriceBest_i = 0   #pridal
        
        self.const1 = const1      #pridal
        self.const2 = const2      #pridal
        self.const3 = const3      #pridal

        for i in range(0,num_dimensions):
            self.velocity_i.append(random.uniform(-1,1))
            self.position_i.append(x0[i])

    # evaluate current fitness
    def evaluate(self,costFunc):
        self.err_i, self.charOrDisch2_i, self.numCharges_i, self.numDischarges_i, self.gridPrice_i, self.numCharChanges_i, self.numBatNullState_i  = costFunc(self.position_i,self.dem,self.pv_gen,self.bat_t0,self.bat_action_t0,self.elprices,self.battery,self.const1,self.const2,self.const3)

        if self.err_i < self.err_best_i or self.err_best_i==-1:
            self.pos_best_i=self.position_i
            self.err_best_i=self.err_i
            
            self.charOrDisch2Best_i = self.charOrDisch2_i         #pridal
            self.numBatNullStateBest_i = self.numBatNullState_i   #pridal
            self.numCharChangesBest_i = self.numCharChanges_i     #pridal
            self.numChargesBest_i = self.numCharges_i             #pridal
            self.numDischargesBest_i = self.numDischarges_i       #pridal
            self.gridPriceBest_i = self.gridPrice_i               #pridal  

    # update new particle velocity
    def update_velocity(self,pos_best_g):
        w=0.5       # constant inertia weight (how much to weigh the previous velocity)
        c1=1        # cognative constant
        c2=2        # social constant

        for i in range(0,num_dimensions):
            r1=random.random()
            r2=random.random()
            vel_cognitive=c1*r1*(self.pos_best_i[i]-self.position_i[i])
            vel_social=c2*r2*(pos_best_g[i]-self.position_i[i])
            self.velocity_i[i]=w*self.velocity_i[i]+vel_cognitive+vel_social

    # update the particle position based off new velocity updates
    def update_position(self,bounds):
        for i in range(0,num_dimensions):
            self.position_i[i]=self.position_i[i]+self.velocity_i[i]
            
            # adjust maximum position if necessary
            if self.position_i[i]>bounds[i][1]:
                self.position_i[i]=bounds[i][1]

            # adjust minimum position if neseccary
            if self.position_i[i] < bounds[i][0]:
                self.position_i[i]=bounds[i][0]

In [5]:
class PSO:
    def __init__(self,costFunc,x0,bounds,num_particles,maxiter,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):
        global num_dimensions

        num_dimensions=len(x0)
        self.err_best_g=-1                   # best error for group
        self.pos_best_g=[]                   # best position for group
        dist_best_g=-1                   #pridal
        thr = 0.2
        dim = len(self.pos_best_g)
        bat_actions = []
        self.grid_actions = []
        self.pv_actions = []
        self.demand_sum = 0
        self.pvToDemand=0
        self.batToDemand=0
        self.gridToDemand=0
        self.grid_total = 0
        self.grid_action = 0
        self.bat_final_act = 1
        
        self.charOrDisch2Best_g = 0      #pridal
        self.numBatNullStateBest_g = 0   #pridal
        self.numCharChangesBest_g = 0    #pridal
        self.numChargesBest_g = 0        #pridal
        self.numDischargesBest_g = 0     #pridal
        
        # establish the swarm
        swarm=[]

        for i in range(0,num_particles):
            swarm.append(Particle(x0,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3))

        # begin optimization loop
        i=0
        while i < maxiter:
            # cycle through particles in swarm and evaluate fitness
            for j in range(0,num_particles):
                swarm[j].evaluate(costFunc)

                if swarm[j].err_i < self.err_best_g or self.err_best_g == -1:
                    self.pos_best_g=list(swarm[j].position_i)
                    self.err_best_g=float(swarm[j].err_i)
                    
                    self.charOrDisch2Best_g = swarm[j].charOrDisch2_i       #pridal
                    self.numBatNullStateBest_g = swarm[j].numBatNullState_i #pridal
                    self.numCharChangesBest_g = swarm[j].numCharChanges_i   #pridal
                    self.numChargesBest_g = swarm[j].numCharges_i           #pridal
                    self.numDischargesBest_g = swarm[j].numDischarges_i     #pridal 


            # cycle through swarm and update velocities and position
            for j in range(0,num_particles):
                swarm[j].update_velocity(self.pos_best_g)
                swarm[j].update_position(bounds)
            i+=1
            
        dim = len(self.pos_best_g)
        bat_actions = []
        self.grid_actions = []
        pv_actions = []
        bat_actions.append(self.pos_best_g[0]-bat_t0)
        self.grid_actions.append(demand[0]-pv_gen[0]+bat_actions[0])
        self.demand_sum = demand[0]
        self.grid_total = self.grid_actions[0]
        for i in range(1,dim):
            bat_actions.append(self.pos_best_g[i]-self.pos_best_g[i-1])
            self.grid_actions.append(demand[i]-pv_gen[i]+bat_actions[i])
            self.demand_sum+=demand[i]
            self.grid_total+=self.grid_actions[i]
            
        for i in range(0,dim):
            if pv_gen[i] >= demand[i]:
                self.pvToDemand += demand[i]
                self.pv_actions.append(demand[i])
            else:
                self.pvToDemand += pv_gen[i]
                self.pv_actions.append(pv_gen[i])
                
            zeroStateRest = demand[i]-pv_gen[i]
            if bat_actions[i]<0 and zeroStateRest>0:
                if abs(bat_actions[i])>=zeroStateRest:
                    self.batToDemand+=zeroStateRest
                else:
                    self.batToDemand+=abs(bat_actions[i])
                    
            if self.grid_actions[i]>0 and pv_gen[i]<demand[i]:
                conRest = demand[i]-pv_gen[i]+bat_actions[i]
                if bat_actions[i] >= 0:
                    self.gridToDemand+=demand[i]-pv_gen[i]
                else:
                    if abs(bat_actions[i])+pv_gen[i] < demand[i]:
                        self.gridToDemand+=conRest
            
            
            
        
        self.grid_action = self.pos_best_g[0] #pridal


In [8]:
class PSOOptimizer():
    def __init__(self, Nparticles, Niter, start, end, windowSize, const1, const2, const3, batCapacity, batSOC, houseNumber):
        global str
        self.Nparticles=Nparticles
        self.Niter=Niter
        self.houseNumber=houseNumber
        self.totalImport = 0
        self.totalExport = 0
        self.start = start
        self.end = end
        self.windowSize=windowSize
        self.bat=Battery(cap=batCapacity, soc=batSOC, act=1, contAct=0, numContChar=0, numContDisch=0, numCharges=1, numDischarges=0, numBatNullState=0, numCharChanges=0)
        self.const1=const1
        self.const2=const2
        self.const3=const3
        
        #polia nakladania s energiou
        self.gridTotalProgress = []
        self.pvToDemandProgress = []
        self.batSOCProgress = []
        self.batActionsProgress = []
        
        self.totalPrice=0
        self.total_demand=0
        self.pvToDemandTotal=0
        
        self.data = pd.read_csv('../data/load/dataid' + str(houseNumber) + '.csv', index_col=0, sep=";")
        self.data.loc[self.data['loadProd'] < 0, 'loadProd'] = 0
        
        self.elprices = pd.read_csv('../data/elprices.csv', index_col=0, sep=",")
        self.elprices = pd.DataFrame(self.elprices[['prices']])
        self.elprices = self.elprices['prices']

        self.iter=0
        self.gen = pd.DataFrame(self.data[['loadProd']])
        self.con = pd.DataFrame(self.data[['loadCons']])
        self.con = self.con['loadCons']
        self.gen = self.gen['loadProd']

    def calculateImportExport(self,gridAction):
        if gridAction >= 0:
            self.totalImport+=gridAction
        if gridAction < 0:
            self.totalExport+=(-gridAction)
            
    def calculatePrice(self,elprices,gridActions):
        price=0
        for i in range(0,len(elprices)):
            if gridActions[i]>0:
                price+=(elprices[i]*gridActions[i])
        return price
    
    def saveResults(self,gridTotalProgress,batSOCProgress,batActionsProgress,pvToDemandProgress,total_demand, pvToDemandTotal,contAct,numContChar,numContDisch,totalPrice,finalnumCharges,finalnumDischarges,finalnumCharChanges,finalnumNullStates):    
        df=self.data[self.start-1:self.end]
        batSoc = batSOCProgress
        if self.iter==0:
            batSoc.pop(0)
            self.iter=1
        dat1 = pd.DataFrame({'newLoadGrid': gridTotalProgress})
        dat2 = pd.DataFrame({'batterySOC': batSoc})
        dat3 = pd.DataFrame({'batteryActions': batActionsProgress})
        dat4 = pd.DataFrame({'pvUsage': pvToDemandProgress})
        dat5 = pd.DataFrame({'prices': self.elprices[self.start-1:self.end]})
        dat5.index = np.arange(0, len(dat5) + 0)
        dat6 = pd.DataFrame({'batteryLastSOC': [self.bat.SOC]})
        dat7 = pd.DataFrame({'totalPrice': [totalPrice]})
        dat8 = pd.DataFrame({'totalImport': [self.totalImport]})
        dat9 = pd.DataFrame({'totalExport': [self.totalExport]})
        dat10 = pd.DataFrame({'totalCons': [total_demand]})
        dat11 = pd.DataFrame({'totalPvUsage': [pvToDemandTotal]})
        dat12 = pd.DataFrame({'contAct': [contAct]})
        dat26 = pd.DataFrame({'batteryAct': [self.bat.action]})
        dat13 = pd.DataFrame({'numContChar': [numContChar]})
        dat14 = pd.DataFrame({'numContDisch': [numContDisch]})
        
        dat15 = pd.DataFrame({'charges': [finalnumCharges]})
        dat16 = pd.DataFrame({'discharges': [finalnumDischarges]})
        dat17 = pd.DataFrame({'CharChanges': [finalnumCharChanges]})
        dat18 = pd.DataFrame({'batNullStates': [finalnumNullStates]})
        
        dat19 = pd.DataFrame({'batLifeLoss': [self.bat.batLifeLoss]})
        if finalnumDischarges==0:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [0]})
        else:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [self.bat.batLifeLoss/finalnumDischarges]})
        dat21 = pd.DataFrame({'avgDOD': [self.bat.avgDOD]})
        dat22 = pd.DataFrame({'avgDODPerDisch': [self.bat.avgDODPerDisch]})
        dat23 = pd.DataFrame({'avgDischAction': [self.bat.avgDischAction]})
        dat24 = pd.DataFrame({'DischAction': [self.bat.avgDischAction*finalnumDischarges]})
        dat25 = pd.DataFrame({'avgSOC': [self.bat.avgSOC]})
        dat27 = pd.DataFrame({'totalBat': [self.bat.totalBat]})
        
        dat28 = pd.DataFrame({'sumDOD': [self.bat.sumDOD]})
        dat29 = pd.DataFrame({'sumDODPerDisch': [self.bat.sumDODPerDisch]})
        dat30 = pd.DataFrame({'sumSOC': [self.bat.sumSOC]})
        dat31 = pd.DataFrame({'sumDischAction': [self.bat.sumDischAction]})
        
        dat1 = pd.concat([dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8, dat9, dat10,dat11, dat12, dat26, dat13, dat14, dat15, dat16, dat17, dat18, dat19, dat20, dat21, dat22, dat23, dat24, dat25, dat27, dat28, dat29, dat30, dat31], axis=1)
        dat1.index = np.arange(self.start, len(dat1) + self.start)
        df = pd.concat([df, dat1], axis=1)
        df.to_csv("Vysledky/PSOMaxBatLife/PSOMaxBatLifeData"+str(self.houseNumber)+".csv",sep=";")
        
    def calculateContCharDisch(self,batSOCProgress,batSoc,bat):
        conWindow=[]
        genWindow=[]
        elprices=[]
        for i in range(self.start,self.end+1):
            conWindow.append(self.con[i])
            genWindow.append(self.gen[i])
            elprices.append(self.elprices[i])
        vvar, vcontAct, vnumContChar,vnumContDisch, vvar3 = contBatActions(batSOCProgress,conWindow,genWindow,batSoc,elprices,bat,self.const1, self.const2)
        self.bat.numContChar = vnumContChar
        self.bat.numContDisch = vnumContDisch
        print('cont actions ',vcontAct, vnumContChar,vnumContDisch)
        
    def calculateBatteryChargingActions(self, Algorithm, conWindow, genWindow, elprices):
        #vypocet kontinualnych nabiti
        var1, charOrDisch, numContChar, numContDisch, var3 = contBatActions2(Algorithm.pos_best_g,conWindow, genWindow, self.bat.SOC, elprices,self.bat,self.const1, self.const2)
        self.bat.contAct=charOrDisch
            
        self.bat.action=Algorithm.charOrDisch2Best_g
        self.bat.numCharChanges+=Algorithm.numCharChangesBest_g
        self.bat.numCharges+=Algorithm.numChargesBest_g
        self.bat.numDischarges+=Algorithm.numDischargesBest_g
    
    def calculateBatteryLossActions(self,batActionsProgress,batSOCProgress,charOrDisch):
        numCharChanges = 0
        numCharges = 1
        numDischarges = 0
        numNullStates=0
        batLifeLoss=0
        dischAction=0
        dischFinish=0
        DODProgress=[]
        DODProgress2=[]
        DOD=0
        avgDOD=0
        avgDOD2=0
        avgSOC=0
        avgDischAction=0
        length = len(batActionsProgress)
        for i in range(0,length):
            batAction=batActionsProgress[i]
            avgSOC+=(batSOCProgress[i]/self.bat.capacity)
            avgDOD2+=(self.bat.capacity-batSOCProgress[i])
            DODProgress2.append((self.bat.capacity-batSOCProgress[i]))
            
            if charOrDisch==1 and batAction < 0:
                numCharChanges+=1
                numDischarges+=1
                charOrDisch=-1
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction=batAction
                dischFinish=0
                
            elif charOrDisch == -1 and batAction < 0:
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction+=(batAction)
                dischFinish=0
                
            elif charOrDisch==-1 and batAction > 0:
                numCharChanges+=1
                numCharges+=1
                charOrDisch=1
                batLifeLoss+=(abs(dischAction)*DOD)
                avgDischAction+=abs(dischAction)
                avgDOD+=DOD
                DODProgress.append(DOD)
                dischFinish=1
                
            elif batAction == 0:
                numNullStates+=1
                
        if dischFinish==0:
            batLifeLoss+=(abs(dischAction)*DOD)
            avgDOD+=DOD
            DODProgress.append(DOD)
        if numDischarges>0:
            batLifeLoss=batLifeLoss/numDischarges
            avgDOD=avgDOD/numDischarges
            avgDischAction=avgDischAction/numDischarges
        else:
            batLifeLoss=0
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        self.bat.numBatNullState=numNullStates
        print("batLifeLoss ",batLifeLoss*numDischarges)
        #print("avgDOD ",(sum(DODProgress)/len(DODProgress)),len(DODProgress))
        print("avgDOD ",(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity)
        print("avgSOC ",(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity)
        print("avgDischAction ",avgDischAction*numDischarges)
        
        totalBat=0
        for i in batActionsProgress:
            totalBat+=abs(i)
        print('totalBat: ',totalBat)
        
        self.bat.batLifeLoss=batLifeLoss*numDischarges
        self.bat.avgDOD=(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity
        self.bat.avgDODPerDisch=(sum(DODProgress)/len(DODProgress))
        self.bat.avgSOC=(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity
        self.bat.avgDischAction=avgDischAction
        self.bat.totalBat=totalBat
        
        self.bat.sumDOD=sum(DODProgress2)
        self.bat.sumDODPerDisch=sum(DODProgress)
        self.bat.sumSOC=sum(batSOCProgress)
        self.bat.sumDischAction=avgDischAction*numDischarges
    
    def optimize(self):
        print("OPTIMALIZACIA particles x iter: ",self.Nparticles,self.Niter)
        windowSize=self.windowSize
        start=self.start
        end = self.end
        total_grid = 0
        total_price = 0
        batteryAction=0
        self.batSOCProgress.append(0)

        import time
        zac = int(round(time.time() * 1000))
        print(zac)
        for c in range(start,end,windowSize):
            d=c+windowSize
            initial= []
            genWindow = []
            conWindow = []
            BSSBounds = []
            elprices = []
            print("row number: ",c)	
            for i in range(c,d):
                if i<=end:
                    elprices.append(self.elprices[i])
                    genWindow.append(self.gen[i])
                    conWindow.append(self.con[i])
                    initial.append(0)
                    BSSBounds.append((self.bat.capacity*0.2,self.bat.capacity*0.8))
                    if i % 5 == 0:
                        print(i)   
                    
            psoOpt = PSO(costFunctionBatteryLifetime3,initial,BSSBounds,num_particles=self.Nparticles,maxiter=self.Niter,demand=conWindow,pv_gen=genWindow,bat_t0=self.bat.SOC,bat_action_t0=batteryAction,elprices=elprices, battery=self.bat,const1=self.const1,const2=self.const2,const3=self.const3)
            
            #prerobene
            alg_bat_actions = []                                                
            alg_grid_actions = []
            alg_bat_actions.append(psoOpt.pos_best_g[0]-self.bat.SOC)
            batteryAction=psoOpt.pos_best_g[0]-self.bat.SOC
            self.batActionsProgress.append(batteryAction)
            
            gridActionHour = (conWindow[0]-genWindow[0]+alg_bat_actions[0])
            alg_grid_actions.append(gridActionHour)
            self.calculateImportExport(gridActionHour)
            alg_demand_sum = conWindow[0]
            alg_grid_total = alg_grid_actions[0]
            for k in range(1,len(genWindow)):
                alg_bat_actions.append(psoOpt.pos_best_g[k]-psoOpt.pos_best_g[k-1])
                self.batActionsProgress.append(psoOpt.pos_best_g[k]-psoOpt.pos_best_g[k-1])
                gridActionHour=(conWindow[k]-genWindow[k]+alg_bat_actions[k])
                alg_grid_actions.append(gridActionHour)
                self.calculateImportExport(gridActionHour)
                alg_demand_sum+=conWindow[k]
                alg_grid_total+=alg_grid_actions[k]
            #prerobene
            
            #vypocet spotreby z pv   
            algPvToDemand=0
            for i in range(0,len(genWindow)):
                pvRest=0
                algPvToDemandInHour = 0
                if genWindow[i] >= conWindow[i]:
                    algPvToDemand += conWindow[i]
                    algPvToDemandInHour += conWindow[i]
                    pvRest = genWindow[i]-conWindow[i]
                else:
                    algPvToDemand += genWindow[i]
                    algPvToDemandInHour += genWindow[i]
                if pvRest > 0 and alg_bat_actions[i]>0:
                    if alg_bat_actions[i]>=pvRest:
                        algPvToDemand += pvRest
                        algPvToDemandInHour += pvRest
                    else:
                        algPvToDemand += alg_bat_actions[i]
                        algPvToDemandInHour += alg_bat_actions[i]
                self.pvToDemandProgress.append(algPvToDemandInHour)
            #vypocet spotreby z pv
            
            self.bat.SOC=psoOpt.pos_best_g[len(genWindow)-1]
            l = len(alg_grid_actions)
            for j in range(0,l):
                self.gridTotalProgress.append(alg_grid_actions[j])
                self.batSOCProgress.append(psoOpt.pos_best_g[j])
            self.total_demand += alg_demand_sum
            total_grid += alg_grid_total
            
            price = self.calculatePrice(elprices,alg_grid_actions)
            self.totalPrice += price
            self.pvToDemandTotal+=algPvToDemand
            
            print()
            
            self.calculateBatteryChargingActions(psoOpt, conWindow, genWindow, elprices)
            self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
            self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)
            print("row success: ",c)
            
            self.saveResults(self.gridTotalProgress, self.batSOCProgress, self.batActionsProgress, self.pvToDemandProgress,self.total_demand,self.pvToDemandTotal,self.bat.contAct,self.bat.numContChar,self.bat.numContDisch,self.totalPrice,self.bat.numCharges,self.bat.numDischarges,self.bat.numCharChanges,self.bat.numBatNullState)
        print('FINAL:')
        kon = int(round(time.time() * 1000))
        #print((kon/1000)/60)
        print("minut: ",((kon-zac)/1000)/60)
        print('Total demand: ',self.total_demand)
        print('Total price: ',self.totalPrice)
        print('Total Import: ',self.totalImport)
        print('Total Export: ',self.totalExport)
        print('pv to demand: ',self.pvToDemandTotal)
        print()
                
        
        
        print()
        print("Charging cycles")
        #self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
        
        self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)



In [7]:
po = PSOOptimizer(Nparticles=13000,Niter=200,start=1,end=8760,windowSize=24,const1=100,const2=100,const3=1,batCapacity=13,batSOC=0,houseNumber=744)
po.optimize()




OPTIMALIZACIA particles x iter:  130 200
1590878711090
row number:  1
5
10
15
20

cont actions  1 0 0
numCharChanges  3
numCharges  2
numDischarges  2
numNullStates  19
batLifeLoss  5.222489107836708e-14
avgDOD  0.8
avgSOC  0.20000000000000023
avgDischAction  6.084022174945858e-14
totalBat:  2.6000000000001307
row success:  1
row number:  25
25
30
35
40
45

cont actions  0 0 0
numCharChanges  21
numCharges  11
numDischarges  11
numNullStates  19
batLifeLoss  1.2128975779031346
avgDOD  0.7937230414224914
avgSOC  0.20627695857750852
avgDischAction  1.4000498198090239
totalBat:  5.632243944757837
row success:  25
row number:  49
50
55
60
65
70

cont actions  1 0 0
numCharChanges  41
numCharges  21
numDischarges  21
numNullStates  19
batLifeLoss  1.6079586336548997
avgDOD  0.7952049987624518
avgSOC  0.20479500123754799
avgDischAction  1.9845258207926362
totalBat:  6.619955449786048
row success:  49
row number:  73
75
80
85
90
95

cont actions  1 0 0
numCharChanges  59
numCharges  30
numDis